In [33]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import glob
from src.utils import Data, markdown_html
import matplotlib.pyplot as plt
from bokeh.io import show, save, output_file
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider, BasicTicker)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.palettes import Inferno256
from bokeh.plotting import figure

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
maryland = Data(state='MD')

INFO:Data collector:Loaded geo shape
INFO:Data collector:Retrieved map info
INFO:Data collector:Loaded daily COVID cases
/Users/wckdouglas/codes/covid/src/utils.py:61: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 61 of the file /Users/wckdouglas/codes/covid/src/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(http.content)
INFO:Data collector:Retrieved populations


In [35]:
data = maryland.geo \
    .merge(maryland.zip_map, on ='Zip', how = 'right') \
    .merge(maryland.zip_covid, on ='Zip', how ='left')\
    .merge(maryland.zip_population.assign(Zip = lambda d: d.Zip.astype(int)), on ='Zip')\
    .assign(Date = lambda d: d.Date.fillna(d.Date.max()))\
    .pipe(lambda d: d[d.Date==d.Date.max()])  \
    .filter(['Zip','City','State','Cases','Date','geometry', 'Population']) \
    .assign(Cases = lambda d: d.Cases.fillna(0)) \
    .pipe(lambda d: d[~pd.isnull(d.geometry)]) \
    .assign(per_population = lambda d: d.Cases / d.Population.astype(int) * 1000)
data.head()

,Zip,City,State,Cases,Date,geometry,Population,per_population
3,20910,Silver Spring,MD,110.0,2020-04-17,"POLYGON ((-77.06795 39.02395, -77.06792 39.023...",42868,2.566017
8,20912,Takoma Park,MD,71.0,2020-04-17,"POLYGON ((-77.02669 38.98656, -77.02644 38.986...",26239,2.705896
12,21001,Aberdeen,MD,8.0,2020-04-17,"POLYGON ((-76.26030 39.52271, -76.26023 39.522...",24453,0.327158
14,21005,Aberdeen Proving Ground,MD,0.0,2020-04-17,"POLYGON ((-76.17461 39.49090, -76.17075 39.494...",2723,0.000000
17,21009,Abingdon,MD,16.0,2020-04-17,"POLYGON ((-76.34463 39.48586, -76.34453 39.485...",29997,0.533387


In [61]:
geosource = GeoJSONDataSource(geojson = data.drop('Date', axis=1).to_json())
color_mapper = LinearColorMapper(palette=Inferno256, 
                           low=data.per_population.min(), 
                           high=data.per_population.max())

p1 = figure(title = 'COVID19 in Maryland\n(Cases per 1,000 people)', 
           plot_height = 600,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p1.xgrid.grid_line_color = None
p1.ygrid.grid_line_color = None
p1.axis.visible = False
p1.title.text_font_size = '25pt'

# Add patch renderer to figure.
states = p1.patches('xs','ys', source = geosource,
                   fill_color = {'field':'per_population',
                                'transform':color_mapper},
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)
# Create hover tool
p1.add_tools(HoverTool(tooltips = [('City','@City'),
                                  ('Zip code','@Zip'),
                                  ('Population','@Population'),
                                  ('Cases', '@Cases'),
                                  ('Cases/1k population', '@per_population')]))

#color bar
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                    orientation='horizontal')
p1.add_layout(color_bar, 'below')

In [62]:
ts_data = maryland.zip_covid\
    .merge(maryland.zip_map.filter(['Zip','City']), on = 'Zip')\
    .groupby(['Zip','City'], as_index=False)\
    .apply(lambda d: d.sort_values('Date')\
                   .assign(Cases = lambda d: d.Cases.fillna(method='ffill', axis=0)\
                   .fillna(0)))\
    .reset_index(drop=True) \
    .groupby(['City','Date'], as_index=False)\
    .agg({'Cases':'sum'})\
    .assign(formatted_date = lambda d: d.Date.astype(str)) 
ts_data.head()

,City,Date,Cases,formatted_date
0,Aberdeen,2020-04-15,8,2020-04-15
1,Aberdeen,2020-04-16,8,2020-04-16
2,Aberdeen,2020-04-17,8,2020-04-17
3,Abingdon,2020-04-13,14,2020-04-13
4,Abingdon,2020-04-14,14,2020-04-14


In [63]:
p2 = figure(x_axis_type="datetime", 
            plot_width = 950, 
            x_axis_label='Date',
            y_axis_label = 'Total Cases',
            title = 'Daily cases by city',
            tools='save,pan,box_zoom,reset,wheel_zoom')
p2.xgrid.grid_line_color = None
p2.ygrid.grid_line_color = None
p2.title.text_font_size = '25pt'
p2.xaxis.axis_label = 'Date'
p2.yaxis.axis_label = 'Total Cases'
p2.xaxis.axis_label_text_font_size = "25pt"
p2.xaxis.major_label_text_font_size = "25pt"
p2.yaxis.axis_label_text_font_size = "25pt"
p2.yaxis.major_label_text_font_size = "25pt"


for zip, zip_df in ts_data.groupby('City'):
    source = ColumnDataSource(zip_df)
    p2.line(x='Date',
            y='Cases',
            color = 'grey',
            line_width=4,
            line_alpha=0.8,
            source=source)
    #add tool tips
    hover = HoverTool(tooltips =[
                     ('Date','@formatted_date'),
                     ('Cases','@Cases'),
                     ('City', '@City')])
p2.add_tools(hover)

In [64]:
p = column(p1, p2)
output_file('output.html')
save(p)

INFO:bokeh.io.state:Session output file 'output.html' already exists, will be overwritten.


'/Users/wckdouglas/codes/covid/output.html'

In [65]:
markdown_html('output.html','COVID.html')

INFO:Data collector:Written 84 lines from 85 lines to COVID.html
